# Plateforme Agnostique de Traitement et d'Analyse des Textes
### Carnet d'expérimentation
---

## Sujet : Utilisation des Vectorizer sklearn pour ignore words.

---

## Observations et environnement
---

In [1]:
cd ../..

/Users/fm/Desktop/Work/Patat


In [2]:
import importlib
import pandas as pd
import seaborn as sns

### Chargement des données

In [3]:
article_filename = ''

In [4]:
import patat.db.article_db
article_db = patat.db.article_db.ArticleDb(article_filename)
article_db.df()

,url,title,article,site,author,date_iso,url_h,c_count
0,https://reseauinternational.net/tous-les-jeune...,"Tous les jeunes, portez la nouvelle (russe)",par Pepe Escobar.\nL’OCS à Samarcande et l’Ass...,reseauinternational.net,NaN,2022-09-30T00:00:00,f95a294c94ff76cc9626ae06300a8b38067f89cf,10366.0
1,https://reseauinternational.net/adhesion-a-la-...,Adhésion à la Russie : 93% pour le « oui » dan...,"Dans les régions de Zaporijia et de Kherson, 9...",reseauinternational.net,NaN,2022-09-30T00:00:00,c924dab7ded47578d81c3ae46f8be0964b3c50f1,1607.0
2,https://lemediaen442.fr/onu-le-premier-ministr...,ONU – Le Premier ministre de Nouvelle-Zélande ...,L’argument principal de la ministre est que le...,lemediaen442.fr,NaN,2022-09-29T00:00:00,3d47a59ef99274fd9ee96c209cc2ab41d6e1f6bb,1444.0
3,https://www.francesoir.fr/societe-environnemen...,Compostage humain: les “funérailles vertes” ga...,"Aux États-Unis, les différents gouvernements r...",www.francesoir.fr,Auteur(s)\nFranceSoir,2022-09-28T13:15:00,0c0341a1f5fae820ee307cb54024df6b06a93d85,4040.0
4,https://www.breizh-info.com/2022/09/27/208410/...,Donatello : génie de la Renaissance,"Portrait du sculpteur Donatello (1386-1466), p...",www.breizh-info.com,NaN,2022-09-27T00:00:00,fcce819327d0302c4cf5e3a8a43b54327ffb8e63,1145.0
...,...,...,...,...,...,...,...,...
77906,https://www.lelibrepenseur.org/une-histoire-vi...,Une histoire vieille comme le monde !,Certains veulent des preuves ! Ils ont raison....,www.lelibrepenseur.org,Par\nAdmin,2010-05-04T00:00:00,8c7a6ecf46d47da1912884a0c0a2837c69f7ad19,NaN
77907,https://www.lelibrepenseur.org/actus/,Actus,"Dernière vidéo !\nIl était temps, enfin ! Je v...",www.lelibrepenseur.org,Par\nuser,2007-07-07T00:00:00,d4d5a37a8ff545664bea75664c8d8695e70d0941,NaN
77908,https://www.lelibrepenseur.org/affiche-des-dec...,Affiche des décès et malaises de sportifs de h...,Un nombre inquiétant de sportifs de haut nivea...,www.lelibrepenseur.org,NaN,NaN,a175027896cf79e76dfad6409694fafacf1919b3,NaN
77909,https://www.lelibrepenseur.org/communaute/,Communauté,Notre Réseau Telegram et ses Groupes Secondair...,www.lelibrepenseur.org,NaN,NaN,6ba257c7b15dd68cadf7dc5d6d8c584db0f1792e,NaN


## Expérience
---

### Réduction corpus

In [5]:
article_db._df['text'] = article_db._df['title'] + '\n' + article_db._df['article']

In [6]:
import patat.util.text

In [7]:
# article_db._df['text'] = article_db._df['text'].apply(patat.util.text.get_latin)

In [8]:
df_light = article_db.df()

In [9]:
df_light = df_light[df_light['c_count']<20_000]

In [12]:
df_light = article_db.df().sample(10_000, random_state=42)

In [11]:
df_light = article_db.df()

### Preprocessor

In [13]:
import patat.util.text
preprocessor = patat.util.text.get_latin
#preprocessor=None

### Tokenizer

In [14]:
from nltk.tokenize import word_tokenize, wordpunct_tokenize, sent_tokenize
from patat.ml.lex_analyser import LexAnalyser

In [15]:
lex = LexAnalyser()

In [16]:
def toklematize(text):
    tokens = lex.get_lemmas_from_text(text)
    return tokens   

In [17]:
tokenizer = toklematize
#tokenizer = word_tokenize
#tokenizer = lex.get_words


### CountVectorizer

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

In [38]:
%%time
vectorizer = CountVectorizer(lowercase=True, preprocessor=preprocessor, tokenizer=tokenizer)
#vectorizer = CountVectorizer(lowercase=True,token_pattern=r'[^_\d\W][^_\d\W]+')
count_matrix = vectorizer.fit_transform(df_light['text'])
count_array = count_matrix.toarray()
df_count = pd.DataFrame(data=count_array,columns = vectorizer.get_feature_names_out())
df_count.shape

CPU times: user 3.68 s, sys: 574 ms, total: 4.26 s
Wall time: 4.36 s


(10000, 105548)

In [41]:
%%time
occ_rares = 10
df_count_sum = pd.DataFrame(df_count.sum())
df_count_sum[df_count_sum[0]<occ_rares].shape

CPU times: user 469 ms, sys: 41.1 ms, total: 510 ms
Wall time: 505 ms


(85273, 1)

In [42]:
keep_words = df_count_sum[df_count_sum[0]>occ_rares].index
len(keep_words)

19188

In [46]:
keep_words

Index(['aa', 'aah', 'aaron', 'ab', 'abad', 'abaissement', 'abaisser',
       'abandon', 'abandonner', 'abasourdi',
       ...
       'ôter', 'œcuménique', 'œil', 'œillères', 'œsophage', 'œuf', 'œufs',
       'œuvre', 'œuvrer', 'œuvres'],
      dtype='object', length=19188)

### TfIdfVectorizer

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [47]:
%%time
vectorizer = TfidfVectorizer(lowercase=True, preprocessor=preprocessor, tokenizer=tokenizer)
#vectorizer = CountVectorizer(lowercase=True,token_pattern=r'[^_\d\W][^_\d\W]+')
count_matrix = vectorizer.fit_transform(df_light['text'])
count_array = count_matrix.toarray()
df_tf = pd.DataFrame(data=count_array,columns = vectorizer.get_feature_names_out())
df_tf.shape

CPU times: user 3.68 s, sys: 694 ms, total: 4.38 s
Wall time: 4.87 s


(10000, 105548)

In [50]:
%%time
occ_rares = 0.4
df_tf_sum = pd.DataFrame(df_tf.sum())
df_tf_sum[df_tf_sum[0]>occ_rares].shape

CPU times: user 1.52 s, sys: 121 ms, total: 1.64 s
Wall time: 1.64 s


(20771, 1)

In [52]:
keep_words = df_tf_sum[df_tf_sum[0]>occ_rares].index
len(keep_words)

20771

In [53]:
df = df_tf[keep_words]

In [54]:
len(df.keys())

20771

In [55]:
df

,aa,aah,aaron,ab,abad,abaissement,abaisser,abandon,abandonner,abasourdi,...,ôter,ōmura,œcuménique,œil,œsophage,œuf,œufs,œuvre,œuvrer,œuvres
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001411,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
9996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
9997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
9998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


## Mesure et observation des résultats
---

## Sauvegarde des résultats
---

## Conclusions
---

- CountVectorizer avec LexAnalyser en préprocessor et tokenizer = OK

## Bricolages
---

In [ ]:
s = df.iloc[200:2000].sum()
df[s.sort_values(ascending=False).index]

In [ ]:
regex = r'[^\d\W][\w]+'

In [ ]:
text=df_light['text'].iloc[0]

In [ ]:
lex.get_lemmas_from_text(text)

In [ ]:
text